Creating a class that samples from a pdf of match outcomes.

In [1]:
import collections
import random
import axelrod as axl
import itertools
import tqdm
from approximate_moran import *

In [2]:
def sample_match_outcomes(players, turns, repetitions):
    """
    Play all matches between pairs of players and return a dictionary mapping player names to a Pdf instance
    """
    matchups = itertools.chain(itertools.combinations(range(len(players)), 2),
                               ((i, i) for i, _ in enumerate(players)))
    match_outcomes = {}
    
    for index_pair in tqdm.tqdm(matchups):
        outcomes = []
        
        for _ in range(repetitions):
            match = axl.Match((players[index_pair[0]], players[index_pair[1]]), turns=turns)
            match.play()
            outcomes.append(match.final_score_per_turn())
            
        counts = collections.Counter(outcomes)
        pdf = Pdf(counts)
        
        player_names = tuple([str(players[i]) for i in index_pair])
        match_outcomes[player_names] = pdf
        
    return match_outcomes

Creating the match outcomes:

In [3]:
players = [s() for s in axl.basic_strategies]

turns = 200
repetitions = 2 # All deterministic strategies but doing 2 anyway (to show that probabilities are all 1.0)

axl.seed(0)
match_outcomes = sample_match_outcomes(players, turns=turns, repetitions=repetitions)
match_outcomes

{('Alternator',
  'Alternator'): Sample space: ((2.0, 2.0),) - Probabilities: [1.0],
 ('Alternator',
  'Anti Tit For Tat'): Sample space: ((2.0, 2.0),) - Probabilities: [1.0],
 ('Alternator',
  'Bully'): Sample space: ((1.985, 2.01),) - Probabilities: [1.0],
 ('Alternator',
  'Cooperator'): Sample space: ((4.0, 1.5),) - Probabilities: [1.0],
 ('Alternator',
  'Cycler DC'): Sample space: ((2.5, 2.5),) - Probabilities: [1.0],
 ('Alternator',
  'Defector'): Sample space: ((0.5, 3.0),) - Probabilities: [1.0],
 ('Alternator',
  'Suspicious Tit For Tat'): Sample space: ((2.5, 2.5),) - Probabilities: [1.0],
 ('Alternator',
  'Tit For Tat'): Sample space: ((2.515, 2.49),) - Probabilities: [1.0],
 ('Alternator',
  'Win-Shift Lose-Stay'): Sample space: ((2.25, 2.25),) - Probabilities: [1.0],
 ('Alternator',
  'Win-Stay Lose-Shift'): Sample space: ((2.25, 2.25),) - Probabilities: [1.0],
 ('Anti Tit For Tat',
  'Anti Tit For Tat'): Sample space: ((2.0, 2.0),) - Probabilities: [1.0],
 ('Anti Tit Fo

Compare the Moran process to the approximated moran process.

In [4]:
def update_fixation_count(moran_process, dictionary):
    """
    Play a moran process and update the 
    dictionary counting the winners
    """
    moran_process.play()
    try:
        dictionary[moran_process.winning_strategy_name] += 1
    except KeyError:
        dictionary[moran_process.winning_strategy_name] = 1

        
approx_fixation_counts = {}
exact_fixation_counts = {}
reps = 100

for seed in range(reps):
    axl.seed(seed)
    amp = ApproximateMoranProcess(players, match_outcomes)
    update_fixation_count(amp, approx_fixation_counts)
    
    axl.seed(seed)
    mp = axl.MoranProcess(players, turns=turns)
    update_fixation_count(mp, exact_fixation_counts)

        
approx_fixation_probabilites = {k: v / reps for k, v in approx_fixation_counts.items()}
exact_fixation_probabilites = {k: v / reps for k, v in exact_fixation_counts.items()}

We see that we get the same fixation probabilities:

In [5]:
for key, value in approx_fixation_probabilites.items():
    print("{}: approximate={} exact={}".format(key, value, exact_fixation_probabilites[key]))

Suspicious Tit For Tat: approximate=0.03 exact=0.03
Bully: approximate=0.1 exact=0.09
Tit For Tat: approximate=0.15 exact=0.13
Cycler DC: approximate=0.07 exact=0.12
Defector: approximate=0.31 exact=0.31
Anti Tit For Tat: approximate=0.02 exact=0.01
Win-Shift Lose-Stay: approximate=0.08 exact=0.07
Win-Stay Lose-Shift: approximate=0.12 exact=0.12
Cooperator: approximate=0.01 exact=0.01
Alternator: approximate=0.11 exact=0.11
